# Boresight determination for the extended grid images

## IMPORT all needed packages

In [1]:
import numpy as np 
from astropy.io import fits

from photutils.detection import DAOStarFinder, IRAFStarFinder, findstars
from photutils.psf import DAOGroup
from photutils.psf import IntegratedGaussianPRF
from photutils.background import MMMBackground
from photutils.background import MADStdBackgroundRMS
from astropy.modeling.fitting import LevMarLSQFitter
from astropy.stats import gaussian_sigma_to_fwhm

from photutils.psf import IterativelySubtractedPSFPhotometry
from astropy.stats import sigma_clipped_stats
import pandas as pd
import re
from pathlib import Path


## Define needed parameters for all files

This is the only place you should need to change anything in. You need to have this code in the folder above your data folders. If that's not the case then, you might need to fiddle with the code to make it work.


### cooldown 
should be the number of the cooldown. This will in the next section define a path to your data folder. Your main data folder per cooldown should look like LB076 where the last 3 numbers are the cooldown (e.g. 076).

### pv
This is the date of the reduction. Or whatever you want to use to separate between different tries. This is added to the excel filename, so if you don't want to overwrite your old file, change this. If you are doing several versions during one day, add a modifier. This is a string, so it doesn't need to be a number. So adding v1 for example will work.

### fileStart
This is the number of the first file the code considers. If you know you have files that are not good in your set, you can define the range starting from this. This code isn't sophisticated enough to handle a gap between files, for this you need to change the code, include the bad files, or do the analysis in 2 or more parts and combine the results by hand. 

### fileEnd
This is the number of the last file the code considers. If you know you have files that are not good in your set, you can define the range ending to this.

### sigma_psf
You shouldn't need to change this. This is used in the FWHM calculation and it's an estimation that generally works. I would keep it the same so that the results are comparable

### max_peak
This is to avoid code detecting artefacts. It defines a maximum value for any pixel it considers. So if the value is over this, it doesn't consider it a valid center point. The current value should be ok, but you might need to change it, so it was left here.

### last_update
This adds to the sibspos.ini file comment row. You should put your initials and the date so we know when this was updated. I also added the number of the cooldown because then it's easy to find the data that was used to produce the code. Remember to add this to the instrument computer also!

In [2]:
cooldown = '078'
pv = '20200326' 
fileStart = 0
fileEnd = 100
cd='LB'+cooldown

sigma_psf = 2.35
max_peak = 8000.

my_path='Cooldown_Results/Cooldown_78/'

all_pixels_file=(my_path+'FORCAST_all_pixels_%s_%s.xlsx' %(cd, pv))
bs_pixels_file=(my_path+'FORCAST_pixels_%s_%s.xlsx' %(cd, pv))

last_update = 'AP 03/26/2020 CD78'

## Add the paths to the files into a list

In [3]:
b='B/'
r='R/'
blue='LB'+cooldown+'/'+b
red='LB'+cooldown+'/'+r

file_list = []

rootdirB = Path(blue)
rootdirR = Path(red)

# Return a list of regular files only, not directories
file_listB = [f for f in sorted(rootdirB.glob('**/*')) if f.is_file()]
file_listR = [f for f in sorted(rootdirR.glob('**/*')) if f.is_file()]


for line in file_listB:
    file_list.append(line)
for line in file_listR:
    file_list.append(line)

mm_files = []

for row in file_list:
    if row.parts[2].startswith('.'):
        continue
    else:
        numbers = re.findall('\d+', row.parts[2])
        if(int(numbers[1]) > fileStart and int(numbers[1]) < fileEnd):
            file = fits.open(row)
            if (file[0].header['APERTURE'] == 'extended grid 13-12'):
                if (file[0].header['FILTER'] == 'dark'):
                    continue
                else:
                    mm_files.append(row)

print(mm_files)

[PosixPath('LB078/B/bLB078_0034.fits'), PosixPath('LB078/B/bLB078_0035.fits'), PosixPath('LB078/B/bLB078_0036.fits'), PosixPath('LB078/B/bLB078_0037.fits'), PosixPath('LB078/B/bLB078_0038.fits'), PosixPath('LB078/B/bLB078_0039.fits'), PosixPath('LB078/B/bLB078_0040.fits'), PosixPath('LB078/B/bLB078_0041.fits'), PosixPath('LB078/B/bLB078_0050.fits'), PosixPath('LB078/B/bLB078_0051.fits'), PosixPath('LB078/B/bLB078_0052.fits'), PosixPath('LB078/B/bLB078_0053.fits'), PosixPath('LB078/B/bLB078_0054.fits'), PosixPath('LB078/B/bLB078_0055.fits'), PosixPath('LB078/B/bLB078_0056.fits'), PosixPath('LB078/B/bLB078_0057.fits'), PosixPath('LB078/R/rLB078_0042.fits'), PosixPath('LB078/R/rLB078_0043.fits'), PosixPath('LB078/R/rLB078_0044.fits'), PosixPath('LB078/R/rLB078_0045.fits'), PosixPath('LB078/R/rLB078_0046.fits'), PosixPath('LB078/R/rLB078_0047.fits'), PosixPath('LB078/R/rLB078_0048.fits'), PosixPath('LB078/R/rLB078_0049.fits'), PosixPath('LB078/R/rLB078_0050.fits'), PosixPath('LB078/R/rLB07

In [4]:
forcast_boresight = pd.DataFrame(columns=['filename','aperture','dichroic','instcfgs','wavelength','x','y','fwhm','x_offset','y_offset'])

## Open and extract information on each file and save it to dataframe

In [5]:
writer = pd.ExcelWriter(all_pixels_file)
for row in mm_files:
    all_positions=pd.DataFrame(columns=['x','y','fwhm'])
    filename = row
    
    file = fits.open(filename)
    file_map=fits.PrimaryHDU(data=file[0].data[0])
    file_map.header = file[0].header
    
    wavelength = file[0].header['WAVELNTH']
    dichroic = file[0].header['DICHROIC']
    aperture = file[0].header['APERTURE']
    filter_c = file[0].header['FILTER']
    instcfgs = file[0].header['INSTCFGS']
    chan = file[0].header['DETCHAN']
    
    if (instcfgs in forcast_boresight['instcfgs'] or instcfgs == 'F056_Barr2'): continue
    else:
        col_median = np.median(file_map.data, axis=0)
        file_map.data = file_map.data - col_median
        
        #file_map.writeto('Cooldown_Results/Cooldown_78/New_fits/'+cd+'_'+instcfgs+'.fits')
        
        image = file_map.data
        bkgrms = MADStdBackgroundRMS()
        std = bkgrms(image)
    
        r_hi = 0.5
        r_lo = 0.
        s_hi = 1.
        s_lo = 0.2
        
        iraffind = IRAFStarFinder(threshold=3.5*std, fwhm=sigma_psf*gaussian_sigma_to_fwhm, 
                                  minsep_fwhm=0.01, roundhi=r_hi, roundlo=r_lo, 
                                  sharplo=s_lo, sharphi=s_hi, peakmax=max_peak, exclude_border = True, brightest=132)


        #iraffind = IRAFStarFinder(threshold=3.5*std, fwhm=sigma_psf*gaussian_sigma_to_fwhm, 
        #                          minsep_fwhm=0.01, roundhi=r_hi, roundlo=r_lo, 
        #                          sharplo=s_lo, sharphi=s_hi, peakmax=max_peak, exclude_border = True)
        
        sources = iraffind(image) 
    
        for s, source in enumerate(sources):
            x = np.around(sources[s][1]+1, decimals = 1)
            y = np.around(sources[s][2]+1, decimals = 1)
            fwhm = np.around(sources[s][3], decimals = 2)
            sharp = np.around(sources[s][4], decimals = 2)
            rness = np.around(sources[s][5], decimals = 2)
            peak = np.around(sources[s][9], decimals = 2)
            flux = np.around(sources[s][10], decimals = 2)
            s_coord = pd.Series([x, y, fwhm],index=['x', 'y', 'fwhm'])
            all_positions=all_positions.append(s_coord, ignore_index=True)
 
        
        all_positions.to_excel(writer,instcfgs,index=False)
        writer.save()
        
        find_cy = all_positions[all_positions['y'].between(125, 135)]
        find_cx = find_cy[find_cy['x'].between(115, 135)]

        
        x = find_cx['x'].values[0]
        y = find_cx['y'].values[0]
        fwhm = find_cx['fwhm'].values[0]

        
        xoff = x-128
        yoff = y-128

        fn= filename.parts[2]

    
        bs = pd.Series([fn, aperture, dichroic,instcfgs, wavelength,x,y,fwhm,xoff,yoff],index=['filename', 'aperture','dichroic','instcfgs','wavelength','x', 'y', 'fwhm','x_offset', 'y_offset'])

        forcast_boresight=forcast_boresight.append(bs, ignore_index=True)

writer.close()

## Display and modify the table
Display the table created to be checked visually and modify it to the form to be export it to excel. Finally export the file to excel.

In [6]:
forcast_boresight=forcast_boresight.sort_values(by=['wavelength'])
forcast_boresight = forcast_boresight.reset_index()
del forcast_boresight['index']
display(forcast_boresight)

,filename,aperture,dichroic,instcfgs,wavelength,x,y,fwhm,x_offset,y_offset
0,bLB078_0034.fits,extended grid 13-12,Mirror (swc),F056,5.6,129.1,129.5,2.47,1.1,1.5
1,bLB078_0035.fits,extended grid 13-12,Mirror (swc),F064,6.4,128.9,132.4,1.92,0.9,4.4
2,bLB078_0051.fits,extended grid 13-12,Dichroic,F064_Barr2,6.4,129.5,132.5,1.88,1.5,4.5
3,bLB078_0036.fits,extended grid 13-12,Mirror (swc),F077,7.7,128.3,128.8,2.10,0.3,0.8
4,bLB078_0052.fits,extended grid 13-12,Dichroic,F077_Barr2,7.7,129.0,129.1,2.09,1.0,1.1
5,bLB078_0037.fits,extended grid 13-12,Mirror (swc),F088,8.8,129.7,129.5,2.37,1.7,1.5
6,bLB078_0053.fits,extended grid 13-12,Dichroic,F088_Barr2,8.8,130.3,129.8,2.55,2.3,1.8
7,bLB078_0039.fits,extended grid 13-12,Mirror (swc),F111,11.1,127.8,128.2,2.14,-0.2,0.2
8,bLB078_0055.fits,extended grid 13-12,Dichroic,F111_Barr2,11.1,128.3,128.3,2.16,0.3,0.3
9,bLB078_0038.fits,extended grid 13-12,Mirror (swc),F112,11.2,129.0,129.3,2.48,1.0,1.3


In [7]:
cd='LB'+cooldown

writer = pd.ExcelWriter(bs_pixels_file)
forcast_boresight.to_excel(writer,'Sheet1',index=False)
writer.save()

## Produce the sibspos.ini file

The following code will make a sibspos.ini file. It sorts the filters so that they are in the order of the file and then adds all the necessary comment rows. After it goes through the forcast_boresight dataframe and extracts the pixel positions.

In [9]:
filter_order_swc = ['F056', 'F064', 'F064_Barr2', 'F077', 'F077_Barr2', 'F088', 'F088_Barr2', 
        'F111', 'F111_Barr2', 'F112', 'F112_Barr2', 'F197', 'F197_Barr2', 'F253', 'F253_Barr2']

filter_order_lwc = ['F113', 'F118', 'F242', 'F315', 'F315_Barr2', 'F336', 'F336_Barr2', 'F348', 
                   'F348_Barr2', 'F371', 'F371_Barr2']                    

textList = ['# Last	updated: '+last_update, '# SIBS	image	offsets (pinhole)', '#   SPECTEL1	dx	dy']

for swc in filter_order_swc:
    if swc[-5:] == 'Barr2':
        channel = 'FOR_'+swc.split('_')[0]+'B2'
    else:
        channel = 'FOR_'+swc

    find_wl = forcast_boresight[forcast_boresight['instcfgs'] == swc]
    x = find_wl['x'].values[0]
    y = find_wl['y'].values[0]
    text = 'swc '+channel+'	'+str(x)+' '+str(y)

    textList.append(text)
    
textList.append('#   SPECTEL2	dx  dy')

for lwc in filter_order_lwc:
    if lwc[-5:] == 'Barr2':
        channel = 'FOR_'+lwc.split('_')[0]+'B2'
    else:
        channel = 'FOR_'+lwc

    find_wl = forcast_boresight[forcast_boresight['instcfgs'] == lwc]
    x = find_wl['x'].values[0]
    y = find_wl['y'].values[0]
    text = 'lwc '+channel+'	'+str(x)+' '+str(y)

    textList.append(text)

sibspos = open(my_path+'sibspos.ini', 'w+')

for line in textList:
  # write line to output file
  sibspos.write(line)
  sibspos.write("\n")
sibspos.close()